In [29]:
!pip install langchain pypdf pymongo PyPDF2 boto3 botocore dotenv tabula-py


  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [75 lines of output]
  C:\Users\karol\Desktop\python-projects\pytorch-start\env\Lib\site-packages\setuptools\installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
    warnings.warn(
    error: subprocess-exited-with-error
  
    python setup.py egg_info did not run successfully.
    exit code: 1
  
    [17 lines of output]
    Traceback (most recent call last):
      File "<string>", line 2, in <module>
      File "<pip-setuptools-caller>", line 14, in <module>
      File "C:\Users\karol\AppData\Local\Temp\pip-wheel-5ernexcl\distribute_1bcbf0477854411281825da787fdf69e\setuptools\__init__.py", line 2, in <module>
        from setuptools.extension import Extension, Library
      File "C:\Users\karol\AppData\Local\Temp\pip-wheel-5ernexcl\distribute_1bcbf0477854411281825da787fdf69e\setuptools\

In [3]:
!pip install tabula-py pdfplumber langchain langchain-core langchain-openai langchain_community

     ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
      --------------------------------------- 0.0/2.3 MB 653.6 kB/s eta 0:00:04
     - -------------------------------------- 0.1/2.3 MB 1.1 MB/s eta 0:00:03
     --- ------------------------------------ 0.2/2.3 MB 1.3 MB/s eta 0:00:02
     ------- -------------------------------- 0.4/2.3 MB 2.2 MB/s eta 0:00:01
     ------------- -------------------------- 0.7/2.3 MB 3.2 MB/s eta 0:00:01
     ----------------- ---------------------- 1.0/2.3 MB 3.5 MB/s eta 0:00:01
     ------------------- -------------------- 1.1/2.3 MB 3.6 MB/s eta 0:00:01
     ------------------------ --------------- 1.4/2.3 MB 3.7 MB/s eta 0:00:01
     ---------------------------- ----------- 1.6/2.3 MB 3.8 MB/s eta 0:00:01
     ------------------------------- -------- 1.8/2.3 MB 4.0 MB/s eta 0:00:01
     ----------------------------------- ---- 2.0/2.3 MB 3.9 MB/s eta 0:00:01
     ---------------------------------------  2.3/2.3 MB 4.1 MB/s eta

In [27]:

import sys
import PyPDF2
import re

In [4]:
from DocumentDB import DocumentDB

documentDB = DocumentDB()

client = documentDB.build_client(True)

In [5]:
DATABASE_NAME = "legal_acts"

In [8]:
db = client[DATABASE_NAME]

In [9]:
users_collection = db['users']

In [10]:
user_id = users_collection.insert_one({
    'username': 'john_doe222',
    'email': 'john.doe@example.com'
}).inserted_id

In [4]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("D20200321Lj.pdf")
pages = loader.load()

In [16]:
!pwd

1055.24s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/home/sagemaker-user/project-main


In [5]:
import pdfplumber


def extract_tables_from_pdf(pdf_path):
    all_tables = []
    current_table = []
    current_table_pages = []
    last_table_page = -1
    data = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            tables = page.extract_tables()
            page_text = page.extract_text()

            new_page_text = re.sub("©\s*Kancelaria\s+Sejmu\s*s\s*\.\s*\d+\s*/\s*\d+\s*", '', page_text).replace("\n",
                                                                                                                " ")
            for table_num, table in enumerate(tables):
                k = str(table[0][0]).replace('\n', ' ')

                table_start_pos = new_page_text.find(str(table[0][0]).replace('\n', ' ')[:20])

                if page_num == last_table_page + 1 and table_num == 0:
                    # Check for intervening text
                    intervening_text = new_page_text[:table_start_pos].strip()
                    if intervening_text:
                        # If there's intervening text, it's a new table
                        if current_table:
                            all_tables.append({
                                "table": current_table,
                                "pages": current_table_pages
                            })
                        current_table = table
                        current_table_pages = [page_num]
                    else:
                        # No intervening text, so it's a continuation
                        current_table.extend(table)
                        current_table_pages.append(page_num)
                elif table and current_table and table[0] == current_table[0]:
                    current_table.extend(table[1:])
                    current_table_pages.append(page_num)
                else:
                    if current_table:
                        all_tables.append({
                            "table": current_table,
                            "pages": current_table_pages
                        })
                    current_table = table
                    current_table_pages = [page_num]
                last_table_page = page_num
        if current_table:
            all_tables.append({
                "table": current_table,
                "pages": current_table_pages
            })
    return all_tables


pdf_path = "legal_acts/D19830207Lj.pdf"
tables = extract_tables_from_pdf(pdf_path)

for i, table in enumerate(tables):
    print(f"Table {i + 1}:")
    for row in table:
        print(row)


Table 1:
table
pages


In [7]:
k = 4

In [ ]:
# prompt for pages with tables

# You will get document page with text and/or with table, you need return normal text and you should summary the tables including all information provided in table, using full sentences and paste this summarization where the table was, return result without any explanation!!! give the plain text without any formation such as indices or new lines
#
# here the text:
#
# 2) ilość leku recepturowego, którego dotyczy odpłatność ryczałtowa, oraz sposób
# obliczania kosztu sporządzania leku recepturowego
# – biorąc pod uwagę dostępność do leków, bezpieczeństwo ich stosowania oraz postać
# farmaceutyczną.
# Art. 7. 1. Ustala się urzędową marżę hurtową w wysokości 6% ceny zbytu netto
# leku, środka spożywczego specjalnego przeznaczenia żywieniowego lub wyrobu
# medycznego, nie niższą niż 50 groszy i nie wyższą niż:
# 1) 150 zł – w przypadku, o którym mowa w art. 6 ust. 1 pkt 1;
# 2) 2000 zł – w przypadkach, o których mowa w art. 6 ust. 1 pkt 2 i 3.
# 2. Przedsiębiorcy prowadzący obrót hurtowy w rozumieniu art. 72 ust. 3 ustawy
# z dnia 6 września 2001 r. – Prawo farmaceutyczne są obowiązani stosować cenę zbytu
# netto wraz z marżą, o której mowa w ust. 1, oraz właściwą stawkę podatku od towarów
# i usług niezależnie od miejsca dostawy leku, środka spożywczego specjalnego
# przeznaczenia żywieniowego lub wyrobu medycznego będącego przedmiotem tego
# obrotu.
# 3. Marża, o której mowa w ust. 1, może być dzielona pomiędzy przedsiębiorców
# prowadzących obrót hurtowy.
# [4. Ustala się urzędową marżę detaliczną naliczaną od ceny hurtowej leku,
# środka spożywczego specjalnego przeznaczenia żywieniowego albo wyrobu
# medycznego stanowiącego podstawę limitu w danej grupie limitowej, w wysokości:
# od do zasada marży
# – 5,00 zł 40%
#           5,01 zł 10,00 zł 2 zł + 30% × (x − 5,00 zł)
# 10,01 zł 20,00 zł 3,50 zł + 20% × (x −10,00 zł)
# 20,01 zł 40,00 zł 5,50 zł + 15% × (x −20,00 zł)
# 40,01 zł 80,00 zł 8,50 zł + 10% × (x −40,00 zł)
# 80,01 zł 160,00 zł 12,50 zł + 5% × (x −80,00 zł)
# 160,01 zł 320,00 zł 16,50 zł + 2,5% × (x − 160,00 zł)
# 320,01 zł 640,00 zł 20,50 zł + 2,5% × (x − 320,00 zł)
# 640,01 zł 1280,00 zł 28,50 zł + 2,5% × (x − 640,00 zł)
# 1280,01 zł 44,50 zł + 1,25% × (x − 1280,00 zł)
# – gdzie x oznacza cenę hurtową leku, środka spożywczego specjalnego przeznaczenia
# żywieniowego albo wyrobu medycznego stanowiącego podstawę limitu,
# uwzględniającą liczbę DDD leku, jednostek środka spożywczego specjalnego

In [38]:
####

# prompt for annotation of changes

# masz za zadanie wyciagnać jesli jest, do którego art, ust, pkt odnosi się ten tekst, i date kiedy ma wejść, i czy ma przepis wejść w życie "add" powinno być true, jeśli uchylenie przepisu to falsedaj odpowiedz w formacie json:
# {
#     "arts":[
#         {
#             "art": "",
#             "usts": [
#                 {
#                     "ust": "",
#                     "pkt": [.....]
#                 }
#             ]
#         }, ....
#     ]
#     "add":boolean
#     date:""
# }
#
# daj tylko wynik!!!, bez tłumaczeń :
#
#
#
# ia. Dodane ust. 1a i 1b w art. 34 wejdą w życie z
# dn. 1.07.2024 r.
# (Dz. U. z 2023 r.
# poz. 1938).



In [16]:

import os
from SecretManager import get_secret

OPENAI_KEY = get_secret("OpenAiApiKey", os.getenv('REGION_NAME'))

print(OPENAI_KEY)

{'OPEN_API_KEY': 'sk-proj-43WipF94bBmSt0O2ZdWmT3BlbkFJBe8PXpIiiRW1vkmHckZI'}


In [5]:
from langchain_openai import ChatOpenAI

openAIModel = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=OPENAI_KEY["OPEN_API_KEY"]
)


In [11]:
annotation_of_changes_schema = {
    "title": "AnnotationOfChanges", # Title of the schema, is required to be
    "description": "Schema for extracting annotations of changes from legal texts",
    "type": "object",
    "properties": {
        "arts": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "art": {
                        "type": "string"
                    },
                    "usts": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "ust": {
                                    "type": "string"
                                },
                                "pkt": {
                                    "type": "array",
                                    "items": {
                                        "type": "string"
                                    }
                                }
                            }
                        }
                    }
                }
            }
        },
        "method": {
            "type": "string",
            "enum": ["add", "remove", "change"]
        },
        "date": {
            "type": "string"
        }
    },
}

In [ ]:
table_extraction_schema = {
    "title": "TableExtraction",
    "description": "Schema for extracting tables from legal texts",
    "type": "array",
    "items": {
        "type": "array",
        "items": {
            "type": "string"
        }
    }
}

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.document_transformers.openai_functions import (
    create_metadata_tagger
)

document_transformer = create_metadata_tagger(metadata_schema=annotation_of_changes_schema, llm=openAIModel,
                                              prompt=ChatPromptTemplate.from_template("""
 You will get document page with text and/or with table, you need return normal text and you should summary the tables including all information provided in table, using full sentences and paste this summarization where the table was, return result without any explanation!!! give the plain text without any formation such as indices or new lines

Only extract the properties mentioned in the 'information_extraction' function.

Passage:
{input}
"""))

In [13]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

annotation_examples = [
    {"input": "ia. Dodane ust. 1a i 1b w art. 34 wejdą w życie z dn. 1.07.2024 r. (Dz. U. z 2023 r. poz. 1938).",
     "output": "{\"arts\":[{\"art\":\"34\",\"usts\":[{\"ust\":\"1a\",\"pkt\":[]},{\"ust\":\"1b\",\"pkt\":[]}]}],\"method\":\"add\",\"date\":\"1.07.2024\"}"},
    {"input": "Nowe brzmienie ust. 4 w art. 7 wejdzie w życie z dn. 1.01.2025 r. (Dz. U. z 2023 r. poz. 1938).",
     "output": "{\"arts\":[{\"art\":\"7\",\"usts\":[{\"ust\":\"4\",\"pkt\":[]}]}],\"method\":\"change\",\"date\":\"1.01.2025\"}"}
]

annotation_examples_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)

few_shot_annotation_prompt = FewShotChatMessagePromptTemplate(
    examples=annotation_examples,
    example_prompt=annotation_examples_prompt
)

annotation_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Extract the desired information from the following passage.  Only extract the properties mentioned in the 'information_extraction' function."),
    few_shot_annotation_prompt,
    ("user", "{input}")
])

In [14]:
from langchain.chains.openai_functions import create_structured_output_chain


chain = annotation_prompt | openAIModel.with_structured_output(annotation_of_changes_schema)
# chain = create_structured_output_chain(annotation_of_changes_schema , openAIModel, annotation_prompt, verbose=True)



In [15]:
chain.invoke({"input": "Nowe brzmienie ust. 2 w art. 34 wejdzie w życie z dn. 1.07.2024 r. (Dz. U. z 2023 r. poz. 1938)."})

{'arts': [{'art': '34', 'usts': [{'ust': '2', 'pkt': []}]}],
 'method': 'change',
 'date': '1.07.2024'}

In [40]:
from langchain_core.documents import Document

text = "ia. Dodane ust. 1a i 1b w art. 34 wejdą w życie z dn. 1.07.2024 r. (Dz. U. z 2023 r. poz. 1938)."

documents = [
    Document(
        page_content=text,
    ),
    Document(
        page_content="Nowe brzmienie ust. 4 w art. 7 wejdzie w życie z dn. 1.01.2025 r. (Dz. U. z 2023 r. poz. 1938).",
    )
]
response = document_transformer.transform_documents(documents)

ValueError: Missing some input keys: {'input'}

In [28]:
print(response[0].metadata)
print(response[1].metadata)

{'arts': [{'art': '34', 'usts': [{'ust': '1a', 'pkt': []}, {'ust': '1b', 'pkt': []}]}], 'method': 'add', 'date': '1.07.2024'}
{'arts': [{'art': '7', 'usts': [{'ust': '4', 'pkt': []}]}], 'method': 'add', 'date': '1.01.2025'}


In [ ]:
# gpt-4o-mini

In [45]:
def extract_text_from_pdf(pdf_path):
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(pdf_path)
    text_by_page = []

    # Extract text from each page
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        text_by_page.append(text)

    return text_by_page


def remove_header_footer(text_by_page):
    def clean_first_page_of_legal_act(text):
        return re.split("(?=U\s*S\s*T\s*A\s*W\s*A)", text)[-1]

    def remove_footer_from_page(text):
        split_point = list(re.finditer("\\n\s+\\n", text))
        if split_point:
            last_split = split_point[-1].end()
            text = text[:last_split].strip()
        return text

    cleaned_text_by_page = []
    # Regex pattern to match the header/footer with flexible date
    pattern = 4

    for i in range(0, len(text_by_page)):

        cleaned_text = re.sub("^©\s*Kancelaria\s+Sejmu\s*s\.\s+\d+/\d+\s*\\n\s*\\n\d{4}\s*[-]?\s*\d{2}\s*[-]?\s*\d{2}",
                              '', text_by_page[i])

        if i == 0:
            cleaned_text = clean_first_page_of_legal_act(cleaned_text)
            # cleaned_text = re.split("(U\s*S\s*T\s*A\s*W\s*A\s*).*", cleaned_text)
        r = 4
        cleaned_text = remove_footer_from_page(cleaned_text)
        cleaned_text_by_page.append(cleaned_text)

    # for page_text in text_by_page:
    #     parts = re.split("^©\s*Kancelaria\s+Sejmu\s*s\.\s+\d+/\d+\s*\\n\s*\\n\d{4}\s*[-]?\s*\d{2}\s*[-]?\s*\d{2}", page_text)
    #     cleaned_text = re.sub("^©\s*Kancelaria\s+Sejmu\s*s\.\s+\d+/\d+\s*\\n\s*\\n\d{4}\s*[-]?\s*\d{2}\s*[-]?\s*\d{2}", '', page_text).strip()
    #     cleaned_text_by_page.append(cleaned_text)

    return cleaned_text_by_page


def concatenate_pages(cleaned_text_by_page):
    full_text = ""
    for page in cleaned_text_by_page:
        full_text += page + " "
    return full_text


def remove_new_line(cleaned_text):
    return cleaned_text.replace('\n', ' ')


def find_annotations_of_changes(text_by_page):
    regexes = [
        r'([a-zA-Z.\d\s]{,34}wejd.*\s*w \s*życie\s*z\s*dn\.\s*\d{1,2}\.\d{1,2}\.\d{4}\s*r\.[\sa-zA-Z\.(\d]*\)\s*\.)'
    ]

    matches = []
    for page_text in text_by_page:
        matches.extend(re.findall(regexes[0], page_text))

    return [match.replace("\n", " ") for match in matches]


# text_by_page = extract_text_from_pdf("legal_acts/D19740141Lj.pdf")
# text_by_page = extract_text_from_pdf("legal_acts/D19830207Lj.pdf")
# text_by_page = extract_text_from_pdf("legal_acts/D19910031Lj.pdf")
# text_by_page = extract_text_from_pdf("legal_acts/D20110696Lj.pdf")
text_by_page = extract_text_from_pdf("legal_acts/D20200321Lj.pdf")

l = find_annotations_of_changes(text_by_page)

cleaned_text_by_page = remove_header_footer(text_by_page)
cleaned_legal_act = concatenate_pages(cleaned_text_by_page)
f = 4
cleaned_legal_act = remove_new_line(cleaned_legal_act)

r = 4

In [20]:
cleaned_legal_act

'U S T AWA   z dnia 9  stycznia 2020  r.  o dodatkowym rocznym świadczeniu pieniężnym dla emerytów i  rencistów   Art. 1. Ustawa określa warunki nabywania prawa do dodatkowego rocznego  świadczenia pieniężnego oraz zasady jego wypłaty.   Art. 2. 1. Dodatkowe roczne świadczenie pieniężne przysługuje osobom,  które w  dniu 31  marca roku, w  którym wyp łacane jest dodatkowe roczne  świadczenie pieniężne, mają prawo do:   1) świadczeń, o  których mowa w  art. 2 pkt 1 lit. a i b ustawy z  dnia 29  maja  1974  r. o zaopatrzeniu inwalidów wojennych i  wojskowyc h oraz ich rodzin  (Dz. U. z 2022  r. poz.  2287),   2) świadcz eń z  ubezpieczenia emerytalno -rentowego, o  których mowa  w art. 18 pkt 1–4 ustawy z  dnia 20  grudnia 1990  r. o  ubezpieczeniu  społecznym rolników (Dz.  U. z 2022 r. poz. 933, 1155 i 2140),   3) świadczeń, o  których mowa w  art. 12 i art. 16 ustawy z  dnia 24  stycznia  1991  r. o kombatantach oraz niektórych osobach będących ofiarami represji  wojennych i  okresu p

In [22]:
full_text = cleaned_legal_act

In [6]:
full_text

'U S T AWA   z dnia 9  stycznia 2020  r.  o dodatkowym rocznym świadczeniu pieniężnym dla emerytów i  rencistów   Art. 1. Ustawa określa warunki nabywania prawa do dodatkowego rocznego  świadczenia pieniężnego oraz zasady jego wypłaty.   Art. 2. 1. Dodatkowe roczne świadczenie pieniężne przysługuje osobom,  które w  dniu 31  marca roku, w  którym wyp łacane jest dodatkowe roczne  świadczenie pieniężne, mają prawo do:   1) świadczeń, o  których mowa w  art. 2 pkt 1 lit. a i b ustawy z  dnia 29  maja  1974  r. o zaopatrzeniu inwalidów wojennych i  wojskowyc h oraz ich rodzin  (Dz. U. z 2022  r. poz.  2287),   2) świadcz eń z  ubezpieczenia emerytalno -rentowego, o  których mowa  w art. 18 pkt 1–4 ustawy z  dnia 20  grudnia 1990  r. o  ubezpieczeniu  społecznym rolników (Dz.  U. z 2022 r. poz. 933, 1155 i 2140),   3) świadczeń, o  których mowa w  art. 12 i art. 16 ustawy z  dnia 24  stycznia  1991  r. o kombatantach oraz niektórych osobach będących ofiarami represji  wojennych i  okresu p

In [26]:
import re


class Article:
    def __init__(self, art_number, art_text, passages, subpoints):
        self.art_number = art_number
        self.art_text = art_text
        self.passages = passages
        self.subpoints = subpoints


class Passage:
    def __init__(self, passage_number, passage_text, subpoints):
        self.passage_number = passage_number
        self.passage_text = passage_text
        self.subpoints = subpoints


class Subpoint:
    def __init__(self, subpoint_number, subpoint_text, subSubpointsRefs):
        self.subpoint_number = subpoint_number
        self.subpoint_text = subpoint_text
        self.subSubpointsRefs = subSubpointsRefs


def is_up_to_date_piece(text):
    normalized_text = text.replace(" ", "")
    # Check if the normalized text is exactly "(uchylony)", "(pominięte)", or "(pominięty)"
    return normalized_text not in ["(uchylony)", "(uchylona)", "(pominięte)", "(pominięty)"]


class CustomMarkdownSplitter:
    def __init__(self):
        self.article_split_pattern = r'(Art\. \d+[a-zA-Z]?[\s\S]*?)(?=Art\. \d+[a-zA-Z]?|$)'
        self.passage_pattern = r'(?<!Art\.\s)(?<!Art\.\s\d\.\s)(?<!Art\.\s\d\d\.\s)(?<!Art\.\s\d\d\d\.\s)(?=[\sa-zA-Z[<]\d{1,}[a-z]*\.\s[\w(])'

        self.art_number_extractor = r'Art\.\s*(\d*\w*(?:–\d*\w*)?)'
        self.erase_art_from_text = r'Art\.\s*\d*\w*(?:–\d*\w*)?\.\s*'

        self.split_by_subpoint = r'(?=(?<=[;):,])\s+(?=\d+[a-z]*\)))'

        self.split_by_subSubPoint = r'(?=(?<=[;):,])\s*(?=[a-z]+\s*\)))'

        self.get_passage_number_pattern = r'^\s*(\d+[a-z]*)\s*\.\s*'
        self.get_subpoint_number_pattern = r'^\s*(\d+[a-z]*)\s*\)\s*[\w\(\[]+'
        self.erase_subpoint_number_pattern = r'^\s*(\d+[a-z]*)\s*\)\s*'
        self.get_subSubPoint_pattern = r'^\s*([a-z]*)\s*\)\s*'

        self.erase_format_legal_acts_structure_words_from_art_pattern = r'\s*(?=R\s*o\s*z\s*d\s*z\s*i\s*a\s*[łl]|D\s*Z\s*I\s*A\s*[ŁL]|O\s*d\s*d\s*z\s*i\s*a\s*[łl])'

        self.kupa = 0

    def split_text(self, text):
        def erase_redundant_act_text(act_text):
            return re.split(self.erase_format_legal_acts_structure_words_from_art_pattern, act_text)[0]

        article_split = re.findall(self.article_split_pattern, text)

        articles = []

        for split in article_split:

            art_number = re.findall(self.art_number_extractor, split)
            text = re.sub(self.erase_art_from_text, "", split)
            text = erase_redundant_act_text(text)
            if is_up_to_date_piece(text):
                articles.append(self.split_passage({
                    "art_number": art_number[0].strip(),
                    "text_for_split_art": text.strip()
                })
                )

        return articles

    def split_passage(self, article_to_process):
        def remove_redudant_section_sign(passage_list):
            return [item.strip().rstrip("§") if item.rstrip().endswith('§') else item.strip() for item in passage_list]

        def standardize_section_reference_keyword(passage_list):
            return [item.replace('§', 'ust.') for item in passage_list]

        if article_to_process["art_number"] == '4':
            f = 4
        passage_splits = re.split(self.passage_pattern, article_to_process["text_for_split_art"])
        passage_splits = remove_redudant_section_sign(passage_splits)
        passage_splits = standardize_section_reference_keyword(passage_splits)
        article = Article(article_to_process["art_number"], None, [], [])
        for index_split, passage_split in enumerate(passage_splits):
            section_match = re.match(self.get_passage_number_pattern, passage_split)
            if index_split == 0 and not section_match:
                article_to_process, subpoints = self.split_subpoint(article_to_process,
                                                                    {
                                                                        "passage_number": None,
                                                                        "text_for_split_passage": passage_split
                                                                    }
                                                                    )
                article.art_text = article_to_process["art_text"]
                article.subpoints.extend(subpoints)
            elif section_match:
                section_number = section_match.group(1).strip()  # Extract the number part
                cleaned_passage_text = re.sub(self.get_passage_number_pattern, '', passage_split).strip()
                if is_up_to_date_piece(cleaned_passage_text):
                    _, passage = self.split_subpoint(
                        article_to_process,
                        {
                            "passage_number": section_number,
                            "text_for_split_passage": cleaned_passage_text
                        }
                    )
                    article.passages.append(passage)


            else:
                print("ERROR from passage !!!!!!! \n")

        return article

    def split_subpoint(self, article_to_process, passage_to_process):
        sub_points_splits = re.split(self.split_by_subpoint, passage_to_process["text_for_split_passage"])
        sub_points_splits = [sub_point_split.strip() for sub_point_split in sub_points_splits]
        is_nested_in_passage = passage_to_process["passage_number"] is not None

        subpoints = [] if not is_nested_in_passage else None
        passage = Passage(passage_to_process["passage_number"], None, []) if is_nested_in_passage else None

        if article_to_process['art_number'] == '231':
            f = 4
        for sub_points_index, sub_point_split in enumerate(sub_points_splits):
            sub_point_match = re.match(self.get_subpoint_number_pattern, sub_point_split)
            if sub_points_index == 0 and not sub_point_match and is_nested_in_passage:
                passage.passage_text = sub_point_split
            elif sub_points_index == 0 and not sub_point_match:
                article_to_process["art_text"] = sub_point_split
            elif sub_point_match:
                sub_point_number = sub_point_match.group(1).strip()
                cleaned_subpoint_text = re.sub(self.erase_subpoint_number_pattern, "", sub_point_split).strip()
                if is_up_to_date_piece(cleaned_subpoint_text):
                    subpoint = self.create_subpoint(sub_point_number, cleaned_subpoint_text)
                    if is_nested_in_passage:
                        passage.subpoints.append(subpoint)
                    else:
                        subpoints.append(subpoint)
            else:
                print("Error from subpoints!!! \n")
        return article_to_process, passage if is_nested_in_passage else subpoints

    def create_subpoint(self, sub_point_number, subpoint_text):
        sub_sub_points_splits = re.split(self.split_by_subSubPoint, subpoint_text)
        sub_sub_points = []
        final_text = ""
        for subSubSplit in map(str.strip, sub_sub_points_splits):
            sub_sub_match = re.match(self.get_subSubPoint_pattern, subSubSplit)
            if sub_sub_match:
                sub_sub_sign = sub_sub_match.group(1).strip()
                sub_sub_cleaned_text = re.sub(self.get_subSubPoint_pattern, "", subSubSplit).strip()
                if is_up_to_date_piece(sub_sub_cleaned_text):
                    final_text = final_text + " " + subSubSplit
                    sub_sub_points.append(sub_sub_sign)
            else:
                final_text = final_text + " " + subSubSplit

        return Subpoint(sub_point_number, final_text.strip(), sub_sub_points)


# Initialize the custom splitter
custom_splitter = CustomMarkdownSplitter()

arts = custom_splitter.split_text(full_text)

ERROR from passage !!!!!!! 

ERROR from passage !!!!!!! 

ERROR from passage !!!!!!! 

ERROR from passage !!!!!!! 



In [25]:
arts

In [26]:
kk = 'Art. 185. § 1. Społeczną kontrolę przestrzegania prawa pracy, w  tym przepisów \ni zasad bezpieczeństwa i  higieny pracy, sprawuje społeczna inspekcja pracy.  \n§ 2. Organizację, zadania i  uprawnienia społecznej inspekcji pracy oraz zasady \njej współdziałania z  Państwową Inspekcją Pracy i  innymi państwowymi organami \nnadzoru i  kontroli określają odrębne przepisy.  \nRozdział III  \n(zawierający art.  19–21 – uchylony)  \nDZIAŁ DRUGI  \nStosunek pracy  \nRozdział I  \nPrzepisy ogólne  \n'

erase_format_legal_acts_structure_words_from_art_pattern = r'\s*(?=R\s*o\s*z\s*d\s*z\s*i\s*a\s*[łl]|D\s*Z\s*I\s*A\s*[ŁL]|O\s*d\s*d\s*z\s*i\s*a\s*[łl])'
l = re.split(erase_format_legal_acts_structure_words_from_art_pattern, kk)

print(l)
print(len(l))
print(l[0])
print("-------")
print(l[1])
print(l[2])

['Art. 185. § 1. Społeczną kontrolę przestrzegania prawa pracy, w  tym przepisów \ni zasad bezpieczeństwa i  higieny pracy, sprawuje społeczna inspekcja pracy.  \n§ 2. Organizację, zadania i  uprawnienia społecznej inspekcji pracy oraz zasady \njej współdziałania z  Państwową Inspekcją Pracy i  innymi państwowymi organami \nnadzoru i  kontroli określają odrębne przepisy.', '', 'Rozdział III  \n(zawierający art.  19–21 – uchylony)', '', 'DZIAŁ DRUGI  \nStosunek pracy', '', 'Rozdział I  \nPrzepisy ogólne  \n']
7
Art. 185. § 1. Społeczną kontrolę przestrzegania prawa pracy, w  tym przepisów 
i zasad bezpieczeństwa i  higieny pracy, sprawuje społeczna inspekcja pracy.  
§ 2. Organizację, zadania i  uprawnienia społecznej inspekcji pracy oraz zasady 
jej współdziałania z  Państwową Inspekcją Pracy i  innymi państwowymi organami 
nadzoru i  kontroli określają odrębne przepisy.
-------

Rozdział III  
(zawierający art.  19–21 – uchylony)


In [13]:
arts

In [23]:
import re

text = "Art. 12. 1. Dla celów podatku  dochodowego od osób fizycznych dodatkowe \nroczne świadczenie pieniężne stanowi świadczenie, o  którym mowa w  art. 12 ust. 7 \nustawy z  dnia 26  lipca 1991  r. o podatku dochodowym od osób fizycznych (Dz. U. \nz 2021 r. poz. 1128, z późn. zm.2)). \n[2. Do kwoty doda tkowego rocznego świadczenia pieniężnego stosuje się \nodpowiednio art.  66 ust. 1 pkt 16, art. 81 ust.  8 pkt 2 i art. 85 ust. 9–11 ustawy \nz dnia 27  sierpnia 2004  r. o świadczeniach opieki zdrowotnej finansowanych \nze środków publicznych (Dz.  U. z 2021  r. poz.  1285, z późn. zm.3)).]  \n<2. Do kwoty dodatkowego rocznego świadczenia pieniężnego stosuje się \nodpowiednio art.  66 ust. 1 pkt 16, art.  81 ust. 8 pkt 2, art. 83 ust. 1 \ni art. 85 ust. 9–11 ustawy z  dnia 27  sierpnia 2004  r. o świadczeniach opieki \n                                                           \n1) Zmiany tekstu jednolitego wymienionej ustawy zostały ogłoszone w Dz. U. z 2021 r. poz. 2270 \noraz z 2022 r. poz. 1, 66, 1079, 1692, 1700, 1812, 1967 , 2127 i 2140 . \n2) Zmiany tekstu jednolitego wymienionej ustawy zostały ogłoszone w Dz. U. z 2021 r. poz. 1163, \n1243, 1551, 1574, 1834, 1981, 2071, 2105, 2133, 2232, 2269, 2270, 2328, 2376, 2427, 2430 i \n2490 oraz z 2022 r. poz. 1, 24, 64, 138, 501, 558, 583, 646, 655, 830, 872, 1079, 1265, 1301, \n1358, 1459, 1512, 1561, 1692, 1846, 1967, 2014, 2127 i 2180 . \n3) Zmiany tekstu jednolitego wymienionej ustawy zostały ogłoszone w Dz. U. z 2021 r. poz. 1292, \n1559, 1773, 1834, 1981, 2105, 2120, 2232, 2270, 2427 i 2469 oraz z 2022 r. poz. 64, 91, 526, \n583, 65 5, 807, 974, 1002, 1079, 1265, 1352 , 1700, 1855, 2140 i 2180.  Nowe brzmienie \nust. 2 w art. 12 \nwejdzie w życie z \ndn. 1.01.2024 r. \n(Dz. U. z 2023 r. \npoz. 1407). "

# Replace valid split points with a unique delimiter
modified_text = re.sub(r'(?<!Art\. \d+\.\s)(?<=\d\.\s)', '###SPLIT###', text)

# Split the text at the delimiter
split_text = modified_text.split('###SPLIT###')

# Displaying the split text
split_text


In [10]:
!pip install PyPDF2


14069.99s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 21.0 MB/s eta 0:00:00


In [12]:
import PyPDF2


def extract_text_from_pdf(pdf_path):
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(pdf_path)
    text_by_page = []

    # Extract text from each page
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        text_by_page.append(text)

    return text_by_page


def identify_footer(text_by_page):
    footer_candidates = {}

    # Check the last few lines of each page to find repetitive text patterns
    for page_text in text_by_page:
        lines = page_text.split('\n')
        if len(lines) > 5:  # Ensure there are enough lines to check
            footer_line = lines[-1].strip()  # Get the last line
            if footer_line in footer_candidates:
                footer_candidates[footer_line] += 1
            else:
                footer_candidates[footer_line] = 1

    # Identify the most common footer line
    footer_line, count = max(footer_candidates.items(), key=lambda item: item[1])

    # Assume footer is the line that appears on most pages
    if count > 1:
        return footer_line
    return None


def remove_footer(text_by_page, footer_line):
    cleaned_text_by_page = []

    for page_text in text_by_page:
        cleaned_text = page_text.replace(footer_line, '').strip()
        cleaned_text_by_page.append(cleaned_text)

    return cleaned_text_by_page


def main(pdf_path):
    text_by_page = extract_text_from_pdf(pdf_path)
    footer_line = identify_footer(text_by_page)

    if footer_line:
        print(f"Identified footer: '{footer_line}'")
        cleaned_text_by_page = remove_footer(text_by_page, footer_line)
    else:
        print("No repetitive footer found.")
        cleaned_text_by_page = text_by_page

    # Print cleaned text for each page
    for page_num, text in enumerate(cleaned_text_by_page):
        print(f"Page {page_num + 1}:\n{text}\n")


if __name__ == "__main__":
    pdf_path = "D19910024Lj.pdf"
    main(pdf_path)


Identified footer: 'poz. 1429).'
Page 1:
©Kancelaria Sejmu     s. 1/75 
   
2023 -08-10 
  
Dz. U. 1991 Nr 7 poz. 24   
 
 
U S T AWA  
z dnia 20  grudnia 1990  r. 
o ubezpieczeniu społecznym rolników1) 
Rozdział 1 
Przepisy ogólne  
Art. 1. 1. Ubezpieczenie społeczne rolników, zwane dalej „ubezpieczeniem”, 
obejmuje, na zasadach określonych w  ustawie, rolników i  pracujących z  nimi 
domowników oraz pomocników rolnika, którzy:  
1) posiadaj ą obywatelstwo polskie lub  
2) są uprawnieni do wykonywania pracy na terytorium Rzeczypospolitej Polskiej 
na podstawie art.  87 ustawy z  dnia 20  kwietnia 2004  r. o promocji zatrudnienia 
i instytucjach rynku pracy (Dz.  U. z  2022  r. poz.  690, z późn. zm.2)) lub są 
zwolnieni na podstawie przep isów szczególnych z  obowiązku posiadania 
zezwolenia na pracę.  
1a. (uchylony)  
2. W ubezpieczeniu wyodrębnia się:  
1) ubezpieczenie wypadkowe, chorobowe i  macierzyńskie;  
2) ubezpieczenie emerytalno -rentowe.  
Art. 2. 1. Ubezpieczenie realizu